In [ ]:
#crosstabs - ignore

#pd.crosstab(gss_frame_1.yearborn_binned5, gss_frame_1.year, values=gss_frame_1.polviews, aggfunc='mean') #.to_csv('polviews_crosstab.csv')
#pd.crosstab(gss_frame_1.yearborn_binned5, gss_frame_1.year) #.to_csv('yearborn_crosstab.csv')
#when you actually do this include weights

In [14]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 50)

In [15]:
#weighted average function from sources in description text

def wavg(group, avg_name, weight_name):
    """ 
    original source: https://pbpython.com/weighted-average.html
    http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [16]:
gss_frame_1 = pd.read_csv('fp_gss_1.csv') #a pared-down version of the gss data with key variables

gss_frame_1

,Unnamed: 0,year,age,sex,polviews,happy,conrinc,wtssall,natrace,life,trust,dwelown,marital
0,1,1972,23.0,2,NaN,3.0,NaN,0.444600,NaN,NaN,3.0,NaN,5.0
1,2,1972,70.0,1,NaN,3.0,NaN,0.889300,NaN,NaN,1.0,NaN,1.0
2,3,1972,48.0,2,NaN,2.0,NaN,0.889300,NaN,NaN,2.0,NaN,1.0
3,4,1972,27.0,2,NaN,3.0,NaN,0.889300,NaN,NaN,2.0,NaN,1.0
4,5,1972,61.0,2,NaN,2.0,NaN,0.889300,NaN,NaN,2.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64809,64810,2018,37.0,2,4.0,2.0,NaN,0.471499,1.0,2.0,NaN,2.0,3.0
64810,64811,2018,75.0,2,5.0,2.0,11391.25,0.942997,2.0,2.0,NaN,1.0,1.0
64811,64812,2018,67.0,2,4.0,1.0,2453.50,0.942997,NaN,2.0,1.0,NaN,1.0
64812,64813,2018,72.0,1,6.0,1.0,NaN,0.942997,2.0,NaN,1.0,1.0,1.0


In [17]:
gss_frame_1['polviews'].describe()

count    55328.000000
mean         4.100528
std          1.382474
min          1.000000
25%          3.000000
50%          4.000000
75%          5.000000
max          7.000000
Name: polviews, dtype: float64

In [18]:
#DWELOWN: redefining it such that it produces a mean of homeowners among those who had a coherent answer
gss_frame_1['dwelown'] = gss_frame_1['dwelown'].map({1: 'Homeowner', 2: 'Renter', 3: 'Other', 8: 'IDK', 9: 'NoAnswer', 0: 'NotApplicable'})
gss_frame_1['dwelown'] = gss_frame_1['dwelown'].map({'Homeowner': 1, 'Renter': 0, 'Other': 0, 'IDK': None, 'NoAnswer': None, 'NotApplicable': 0})

#MARITAL: creating new var 'been_married'
gss_frame_1['been_married'] = gss_frame_1['marital'].map({1: 1, 2: 1, 3: 1, 4: 1, 5: 0, 9: None})

In [19]:
#coding polviews such that only those with answers on the 1-to-7 scale are included
gss_frame_1
gss_frame_1 = gss_frame_1.dropna(subset=['polviews'])
gss_frame_1 = gss_frame_1[gss_frame_1['polviews'] > 0]
gss_frame_1 = gss_frame_1[gss_frame_1['polviews'] < 8]
gss_frame_1['sex']  = gss_frame_1['sex'].map({1: 'Male', 2: 'Female'}) #translating gender; did not use this
gss_frame_1.sort_values('conrinc', ascending=False)

,Unnamed: 0,year,age,sex,polviews,happy,conrinc,wtssall,natrace,life,trust,dwelown,marital,been_married
51289,51290,2008,41.0,Male,5.0,2.0,434612.415694,0.875490,2.0,1.0,NaN,1.0,1.0,1.0
51540,51541,2008,37.0,Female,2.0,1.0,434612.415694,0.437745,2.0,NaN,1.0,1.0,3.0,1.0
51616,51617,2008,67.0,Female,4.0,2.0,434612.415694,0.875490,2.0,1.0,1.0,NaN,1.0,1.0
51712,51713,2008,43.0,Male,2.0,2.0,434612.415694,2.135340,NaN,2.0,1.0,NaN,1.0,1.0
51721,51722,2008,52.0,Male,6.0,1.0,434612.415694,1.313234,NaN,1.0,2.0,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64804,64805,2018,46.0,Female,4.0,3.0,NaN,0.471499,3.0,2.0,2.0,NaN,3.0,1.0
64806,64807,2018,89.0,Male,4.0,2.0,NaN,0.471499,NaN,NaN,2.0,0.0,2.0,1.0
64808,64809,2018,19.0,Male,4.0,2.0,NaN,1.885994,1.0,2.0,2.0,NaN,5.0,0.0
64809,64810,2018,37.0,Female,4.0,2.0,NaN,0.471499,1.0,2.0,NaN,0.0,3.0,1.0


In [20]:
gss_frame_1['year_of_birth'] = gss_frame_1.year - gss_frame_1.age
gss_frame_1['yearborn_binned5'] = (gss_frame_1.year_of_birth // 5) * 5 #rounds year born to nearest 5-year interval
gss_frame_1['yearborn_binned2'] = (gss_frame_1.year_of_birth // 2) * 2 #rounds year born to nearest 2-year interval

In [21]:
survey_birthcohort_pairs = pd.DataFrame(gss_frame_1.groupby(['year', 'yearborn_binned5']).size().reset_index(name='counts'))
#survey_birthcohort_pairs['age'] = survey_birthcohort_pairs.year - survey_birthcohort_pairs.yearborn_binned5



In [22]:
survey_years = list(set(survey_birthcohort_pairs['year']))
birth_cohorts = list(set(survey_birthcohort_pairs['yearborn_binned5']))

In [23]:
gss_frame_2 = gss_frame_1[gss_frame_1['year'] == 2000]
gss_frame_2.groupby("yearborn_binned5").apply(wavg, "polviews", "wtssall")

yearborn_binned5
1910.0    4.050019
1915.0    4.389985
1920.0    4.360902
1925.0    4.477262
1930.0    4.463152
1935.0    4.381816
1940.0    4.134017
1945.0    4.100478
1950.0    4.029342
1955.0    4.170850
1960.0    4.099193
1965.0    4.142331
1970.0    4.015727
1975.0    3.778269
1980.0    3.939540
dtype: float64

In [24]:
#calculate weighted means of polviews, happiness, etc. and append it to frame

vars_to_mean = ['age', 'polviews', 'happy', 'conrinc', 'dwelown', 'been_married']

for item in vars_to_mean:
    weighted_means_list = []
    for year in survey_years:
        gss_frame_2 = gss_frame_1[gss_frame_1['year'] == year]
        weightmeans = list(gss_frame_2.groupby("yearborn_binned5").apply(wavg, item, "wtssall"))
        for value in weightmeans:
            weighted_means_list.append(value)
    
    varname = item + "_wmean"
    survey_birthcohort_pairs[varname] = weighted_means_list

#then also a median for the income variable 

median_list = []
for year in survey_years:
    gss_frame_2 = gss_frame_1[gss_frame_1['year'] == year]
    weightmeans = list(gss_frame_2.groupby("yearborn_binned5")['conrinc'].median())
    for item in weightmeans:
        median_list.append(item)

survey_birthcohort_pairs['conrinc_median'] = median_list

In [30]:
survey_birthcohort_pairs = survey_birthcohort_pairs.rename(columns={"year": "survey_yr", "yearborn_binned5":"birth_cohort"})
survey_birthcohort_pairs = survey_birthcohort_pairs[survey_birthcohort_pairs['counts'] > 49]
survey_birthcohort_pairs['is2018'] = np.where(survey_birthcohort_pairs['survey_yr'] == 2018, 1, 0)
survey_birthcohort_pairs['been_married_wmean_binned'] = (survey_birthcohort_pairs.been_married_wmean // .1) * .1
survey_birthcohort_pairs['dwelown_wmean_binned'] = (survey_birthcohort_pairs.dwelown_wmean // .1) * .1
survey_birthcohort_pairs['age_wmean_binned'] = (survey_birthcohort_pairs.age_wmean // 5) * 5

survey_birthcohort_pairs

,survey_yr,birth_cohort,counts,age_wmean,polviews_wmean,happy_wmean,conrinc_wmean,dwelown_wmean,been_married_wmean,conrinc_median,is2018,been_married_wmean_binned,dwelown_wmean_binned,age_wmean_binned,relative_youth
3,1974,1900.0,69,72.080285,3.927014,1.605840,640.965240,0.000000,0.948904,1689.0,0,0.9,-0.0,70.0,11.0
4,1974,1905.0,97,67.046401,4.257737,1.618558,9229.583177,0.000000,0.979382,21952.0,0,0.9,-0.0,65.0,10.0
5,1974,1910.0,80,61.581404,4.075579,1.540701,18447.057463,0.000000,0.947673,25329.0,0,0.9,-0.0,60.0,9.0
6,1974,1915.0,107,57.185651,4.362870,1.734174,21651.309299,0.000000,0.987341,30395.0,0,0.9,-0.0,55.0,8.0
7,1974,1920.0,123,51.937283,4.205573,1.662020,24962.767082,0.000000,0.979093,42215.0,0,0.9,-0.0,50.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,2018,1975.0,204,40.912681,4.095470,1.740394,26881.739579,0.389989,0.774161,31545.0,1,0.7,0.3,40.0,5.0
452,2018,1980.0,218,35.828279,3.770654,1.731509,29582.696311,0.291317,0.782613,31545.0,1,0.7,0.2,35.0,4.0
453,2018,1985.0,206,30.877120,3.750012,1.849783,20092.278580,0.301534,0.458315,22782.5,1,0.4,0.3,30.0,3.0
454,2018,1990.0,184,25.937569,3.808294,1.888039,12058.095930,0.133935,0.307154,19277.5,1,0.3,0.1,25.0,2.0


In [35]:
survey_birthcohort_pairs.counts.describe()

count    362.000000
mean     145.577348
std       74.605153
min       50.000000
25%       93.000000
50%      129.000000
75%      176.000000
max      462.000000
Name: counts, dtype: float64

In [31]:
#create "relative youth" variable

rel_youth_list = []
for item in set(survey_birthcohort_pairs['survey_yr']):
    for value in survey_birthcohort_pairs[survey_birthcohort_pairs['survey_yr']==item]['age_wmean'].rank():
        rel_youth_list.append(value)
        
survey_birthcohort_pairs['relative_youth'] = rel_youth_list

In [32]:
survey_birthcohort_pairs.to_csv(r'C:\Users\Zach Gozlan\Documents\Data Visualization\Final Project\\ppol563_final_project\birthcohort_means.csv', index = None)

In [36]:
survey_birthcohort_pairs

,survey_yr,birth_cohort,counts,age_wmean,polviews_wmean,happy_wmean,conrinc_wmean,dwelown_wmean,been_married_wmean,conrinc_median,is2018,been_married_wmean_binned,dwelown_wmean_binned,age_wmean_binned,relative_youth
3,1974,1900.0,69,72.080285,3.927014,1.605840,640.965240,0.000000,0.948904,1689.0,0,0.9,-0.0,70.0,11.0
4,1974,1905.0,97,67.046401,4.257737,1.618558,9229.583177,0.000000,0.979382,21952.0,0,0.9,-0.0,65.0,10.0
5,1974,1910.0,80,61.581404,4.075579,1.540701,18447.057463,0.000000,0.947673,25329.0,0,0.9,-0.0,60.0,9.0
6,1974,1915.0,107,57.185651,4.362870,1.734174,21651.309299,0.000000,0.987341,30395.0,0,0.9,-0.0,55.0,8.0
7,1974,1920.0,123,51.937283,4.205573,1.662020,24962.767082,0.000000,0.979093,42215.0,0,0.9,-0.0,50.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,2018,1975.0,204,40.912681,4.095470,1.740394,26881.739579,0.389989,0.774161,31545.0,1,0.7,0.3,40.0,5.0
452,2018,1980.0,218,35.828279,3.770654,1.731509,29582.696311,0.291317,0.782613,31545.0,1,0.7,0.2,35.0,4.0
453,2018,1985.0,206,30.877120,3.750012,1.849783,20092.278580,0.301534,0.458315,22782.5,1,0.4,0.3,30.0,3.0
454,2018,1990.0,184,25.937569,3.808294,1.888039,12058.095930,0.133935,0.307154,19277.5,1,0.3,0.1,25.0,2.0
